**Classes**

In [1]:
class Person():
    def __init__(self, name, age, color):
        self.name = name
        self.age = age
        self.color = color
    
    def calc_age(self,)

In [4]:
person0 = Person("Abs Sayem", 30, "Brown")
person0.name

'Abs Sayem'